<a href="https://colab.research.google.com/github/bardiaHSZD/TrendPrediction/blob/main/Dynamic_Mode_Decomposition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt install octave
!apt-get update --fix-missing
!apt install octave

In [10]:
%%writefile dmd_test.m

clear all;

%Quick Test of DMD with control.

%Data collection

A = [1.5 0;0 0.1];
x0 = [4;7];
K = [-1];
m = 20;
DataX = x0;
DataU = [0];

B = [1;0];

for j = 1 : m 
   
  DataX(:,j+1) = A * (DataX(:,j)) + B.*(K*DataX(:,j));
  DataU(:,j) = K .* DataX(1,j);
end

%Data matrices
X   = DataX(:,1:end-1);
Xp  = DataX(:,2:end);
Ups = DataU;

%SVD
[U,Sig,V] = svd(X,'econ');

thresh = 1e-10;
r = length(find(diag(Sig)>thresh));

U    = U(:,1:r);
Sig  = Sig(1:r,1:r);
V    = V(:,1:r);

%DMD

A_DMD  = Xp*V*inv(Sig)*U'

%DMDc - B is known 

A_DMDc = (Xp - B*Ups)*V*inv(Sig)*U'


Writing dmd_test.m


In [11]:
!octave dmd_test.m # -W : no window system

octave: X11 DISPLAY environment variable not set
octave: disabling GUI features
A_DMD =

   5.0000e-01  -2.1751e-17
  -7.4096e-17   1.0000e-01

A_DMDc =

   1.5000e+00   2.2711e-16
  -7.4096e-17   1.0000e-01



In [ ]:
%%writefile dmd_full.m

#X   = StateData(:,1:end-1);
#Xp  = StateData(:,2:end);
#Ups = InputData(:,1:end-1);

Omega = [X;Ups];

[U,Sig,V] = svd(Omega,'econ');

thresh = 1e-10;
rtil = length(find(diag(Sig)>thresh));

Util    = U(:,1:rtil); 
Sigtil  = Sig(1:rtil,1:rtil);
Vtil    = V(:,1:rtil); 

[U,Sig,V] = svd(Xp,'econ');

thresh = 1e-10;
r = length(find(diag(Sig)>thresh));

Uhat    = U(:,1:r); 
Sighat  = Sig(1:r,1:r);
Vbar    = V(:,1:r); 

n = size(X,1); 
q = size(Ups,1);
U_1 = Util(1:n,:);
U_2 = Util(n+q:n+q,:);

approxA = Uhat'*(Xp)*Vtil*inv(Sigtil)*U_1'*Uhat;
approxB = Uhat'*(Xp)*Vtil*inv(Sigtil)*U_2';

[W,D] = eig(approxA);

Phi = Xp * Vtil * inv(Sigtil) * U_1'*Uhat * W;